In [1]:
import cv2, os
from tqdm import tqdm
import numpy as np

In [2]:
import tensorflow as tf

import tensorflow as tf
from tensorflow.python.keras.utils.multi_gpu_utils import multi_gpu_model


gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    # Configure which GPU(s) to use
    tf.config.experimental.set_visible_devices(gpus, 'GPU')
    # Enable memory growth for each GPU
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)

In [3]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Input
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [4]:
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
x = base_model.output
x = GlobalAveragePooling2D()(x)
model = Model(inputs=base_model.input, outputs=x)

In [ ]:
path = "dataset/train/"

trainX = []
trainY = []
for classes in tqdm(os.listdir(path)):
    class_ = path + classes
    for images in os.listdir(class_):
        try:
            image_path = class_ + '/' + images
            image = cv2.imread(image_path)
            image = cv2.resize(image,(224,224))
            image = np.expand_dims(image,axis=0)
            features = model.predict(image)[0]
            trainX.append(features)
            trainY.append(class_.split('/')[-1])
        except:
            continue

In [ ]:
trainX = np.array(trainX)
trainY = np.array(trainY)

In [ ]:
path = "dataset/test/"

testX = []
testY = []
for classes in tqdm(os.listdir(path)):
    class_ = path + classes
    for images in os.listdir(class_):
        try:
            image_path = class_ + '/' + images
            image = cv2.imread(image_path)
            image = cv2.resize(image,(224,224))
            image = np.expand_dims(image,axis=0)
            features = model.predict(image)[0]
            testX.append(features)
            testY.append(class_.split('/')[-1])
        except:
            continue

In [ ]:
testX = np.array(testX)
testY = np.array(testY)

In [ ]:
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
trainY = le.fit_transform(trainY)
testY = le.fit_transform(testY)
trainY = to_categorical(trainY, 6)
testY = to_categorical(testY, 6)
print(trainX.shape,testX.shape,trainY.shape,testY.shape)

In [ ]:
import pickle

pickle.dump(trainX, open('trainX.pkl', 'wb'))
pickle.dump(trainY, open('trainY.pkl', 'wb'))
pickle.dump(testX, open('testX.pkl', 'wb'))
pickle.dump(testY, open('testY.pkl', 'wb'))

In [ ]:
import pickle
from sklearnex import patch_sklearn
patch_sklearn()

In [ ]:
trainX  = pickle.load(open('trainX.pkl', 'rb'))
trainY = pickle.load(open('trainY.pkl', 'rb'))
testX = pickle.load(open('testX.pkl', 'rb'))
testY = pickle.load(open('testY.pkl', 'rb'))

In [ ]:
X = np.concatenate([trainX, testX])
y = np.concatenate([trainY, testY])
print(X.shape,y.shape)

In [ ]:
%%time
from sklearn.neural_network import MLPClassifier

model = MLPClassifier(random_state=42, max_iter=256,hidden_layer_sizes=1024,shuffle=True,
                      activation='logistic', solver='adam',learning_rate='adaptive',validation_fraction=0.15,early_stopping=True,verbose=True)
model.fit(X, y)

In [ ]:
pickle.dump(model, open('mlp_model.pkl', 'wb'))

In [5]:
import pickle

mymodel = pickle.load(open('mlp_model.pkl', 'rb'))
classes = open("classes.txt",'r').read().split('\n')[:-1]
print(classes)

['explicit_moderation', 'gore_moderation', 'intoxicants_moderation', 'sfw_moderation', 'suggestive_moderation', 'weapons_moderation']


C:\ProgramData\anaconda3\lib\site-packages\sklearn\base.py:318: UserWarning: Trying to unpickle estimator LabelBinarizer from version 1.1.1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
C:\ProgramData\anaconda3\lib\site-packages\sklearn\base.py:318: UserWarning: Trying to unpickle estimator MLPClassifier from version 1.1.1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [ ]:
path = "dataset/validation/"

validY = []
predY = []
for class_path in tqdm(os.listdir(path)):
    class_ = path + class_path
    for images in os.listdir(class_):
        try:
            image_path = class_ + '/' + images
            image = cv2.imread(image_path)
            image = cv2.resize(image,(224,224))
            image = np.expand_dims(image,axis=0)
            features = model.predict(image)
            #features = np.expand_dims(features,axis=0)
            pred = mymodel.predict(features)[0]
            name = classes[np.argmax(pred)]
            predY.append(name)
            validY.append(class_.split('/')[2])
        except:
            print(image_path)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
len(validY), len(predY), validY[0], predY[0]

In [ ]:
from sklearn.metrics import classification_report
from sklearn.metrics import ConfusionMatrixDisplay

print(classification_report(validY, predY))

In [ ]:
from sklearn.metrics import confusion_matrix

conf_matrix = confusion_matrix(validY, predY)

In [ ]:
sns.heatmap(conf_matrix,
            annot=True,
            fmt='g',
            xticklabels=classes,
            yticklabels=classes,
    )
plt.ylabel('Prediction',fontsize=13)
plt.xlabel('Actual',fontsize=13)
plt.title('Confusion Matrix',fontsize=17)
plt.show()

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

accuracy = accuracy_score(validY, predY)
precision = precision_score(validY, predY,average='weighted')
recall = recall_score(validY, predY,average='weighted')
f1 = f1_score(validY, predY,average='weighted')

# Print the results
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

In [11]:
image = cv2.imread("person.jpg")
image = cv2.resize(image,(224,224))
image = np.expand_dims(image,axis=0)
features = model.predict(image)
pred = mymodel.predict(features)[0]
name = classes[np.argmax(pred)]
print(name)

sfw_moderation
